In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# 1. 옮겨간 새 사이트 주소
url = "https://contents.history.go.kr/front/nh/view.do"

# 2. 긁고 싶은 ID (파라미터)
params = {
    "levelId": "nh_001_0010"
}

# 3. 사람인 척 하기 (헤더 필수!)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

try:
    # 요청 발사!
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status() # 에러 나면 멈춤

    # HTML 파싱
    soup = BeautifulSoup(response.text, 'html.parser')

    # --- [중요] 본문 찾는 부분 ---
    # 개발자 도구로 본문 감싸고 있는 태그의 class 이름을 확인해야 합니다.
    # 보통 공공기관 사이트는 'view_cont', 'con_view', 'txt-wrap' 중 하나입니다.
    # 일단 가장 유력한 후보들로 찾아봅니다.

    content_div = soup.find('div', class_='article') # 후보 1
    if not content_div:
        content_div = soup.find('div', class_='txt-wrap') # 후보 2

    if content_div:
        # 각주(팝업) 제거 로직 (이전과 동일)
        for footnote in content_div.find_all('span', class_='popup-type1'):
            footnote.decompose()
        for footnote in content_div.find_all('a', class_='note_link'): # 혹시 태그가 a라면
            footnote.decompose()

        # 텍스트 추출
        final_text = content_div.get_text(separator="\n", strip=True)
        print("✅ 성공적으로 텍스트를 가져왔습니다!")
        print("-" * 50)
        print(final_text[:500]) # 앞부분 500자 출력
    else:
        print("❌ 본문 태그(div)를 못 찾았습니다.")
        print("개발자 도구(F12) -> Elements 탭에서 본문이 어떤 class 이름인지 확인해주세요!")

except Exception as e:
    print(f"에러 발생: {e}")

✅ 성공적으로 텍스트를 가져왔습니다!
--------------------------------------------------
총설 －한국사의 전개－
Ⅰ. 한국사의 전개를 보는 시각
한국사는 어떻게 전개되어 왔을까. 이에 대해서는 오래 전부터 王朝의 흥망을 기준으로 보는 견해가 행해져 왔다. 한 왕조가 망하고 다른 한 왕조가 일어나는 데에는 그럴만한 이유가 있는 것이므로, 이 같은 소위 王朝史觀은 그것대로 의미가 있는 것이다. 그러나 한편 생각하면, 왕조의 교체에도 불구하고 사회적인 동질성이 유지되는 경우도 있고, 또 같은 왕조 안에서도 사회적인 변화가 발견되는 경우도 있다. 왕조를 기준으로 역사의 전개를 이해하는 데에 만족할 수가 없는 까닭이 이러한 데에 있는 것이다.
왕조 중심의 역사관을 극복하기 위하여 제기된 새로운 방법이 한국사의 전개과정을 古代·中世·近代의 세 시기로 나누어서 이해하는 것이었다. 그런데 이 같은 예들을 살펴보면, 비록 용어는 달라졌지만 실제에 있어서는 왕조 중심의 관점과 별로 다를 바가 없는 것이다. 그래서 중세와 근대 사이에 近世라는 시대를 설정하여 高麗王朝와 구별하며 朝鮮王朝를 


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
import os
from tqdm import tqdm
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# ==========================================
# 1.세션 설정 (서버가 끊어도 5번 다시 붙음)
# ==========================================
session = requests.Session()
# 접속 끊김(ConnectionError), 5xx 에러 시 0.5초 간격으로 최대 5번 재시도
retries = Retry(total=5, backoff_factor=0.5, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0 Safari/537.36"
}

# 데이터 저장할 폴더
if not os.path.exists("history_data_final"):
    os.makedirs("history_data_final")

base_url = "https://contents.history.go.kr/front/nh/view.do"

print(" 1권~52권 전수 조사를 시작합니다.\n")

# ==========================================
# 2. 전권 루프 (1 ~ 52)
# ==========================================
for vol in range(1, 53):
    vol_str = f"{vol:03d}"
    root_id = f"nh_{vol_str}"

    print(f"📘 [{root_id}] 작업 시작...")

    # -------------------------------------------------
    # (A) 지도(ID 리스트) 확보
    # -------------------------------------------------
    # 진입점: 각 권의 메인 페이지 (여기에 전체 하위 ID가 숨어있음)
    try:
        # timeout을 넉넉히 20초 줌
        response = session.get(base_url, params={"levelId": root_id}, headers=headers, timeout=20)
        html_source = response.text

        # 정규표현식으로 nh_00X_... 패턴 싹 긁기
        pattern = f"({root_id}_[0-9_]+)"
        found_ids = re.findall(pattern, html_source)
        all_ids = sorted(list(set(found_ids)))

        # 자기 자신(root_id)은 제외 (목차 페이지일 확률 높음)
        if root_id in all_ids: all_ids.remove(root_id)

        print(f"   -> 지도 확보 완료! ({len(all_ids)}개 챕터)")

        if len(all_ids) == 0:
            print("   ⚠️ ID를 못 찾았습니다. 건너뜁니다.")
            continue

    except Exception as e:
        print(f"   ❌ 지도 확보 중 치명적 에러: {e}")
        continue

    # -------------------------------------------------
    # (B) 본문 크롤링
    # -------------------------------------------------
    dataset = []

    # 진행바 표시
    for target_id in tqdm(all_ids, desc=f"{vol_str}권 수집"):
        try:
            # timeout=10초. 10초 동안 답 없으면 재시도 하거나 넘어감
            res = session.get(base_url, params={"levelId": target_id}, headers=headers, timeout=10)
            soup = BeautifulSoup(res.text, 'html.parser')

            # 님께서 확인하신 그 'article' 클래스!
            article_div = soup.find('div', class_='article')

            if article_div:
                # 제목 추출
                title_tag = soup.find('h3', class_='view_tit')
                if not title_tag:
                    title_tag = soup.find('h4', class_='tit') # 혹시 몰라 예비용

                title = title_tag.get_text(strip=True) if title_tag else "제목 없음"

                # 불필요한 태그(각주, 링크) 삭제
                for tag in article_div.find_all(['sup', 'a']):
                    tag.decompose()

                # 본문 텍스트
                content = article_div.get_text(separator="\n\n", strip=True)

                # 내용이 너무 짧으면(빈 페이지) 저장 안 함
                if len(content) > 30:
                    dataset.append({
                        "id": target_id,
                        "title": title,
                        "text": content,
                        "volume": vol,
                        "source": res.url
                    })

            # 서버에 너무 무리 주지 않게 0.05초 휴식
            time.sleep(0.05)

        except Exception as e:
            # 하나 실패해도 쿨하게 넘어감 (전체 중단 X)
            # print(f"에러({target_id}): {e}")
            pass

    # -------------------------------------------------
    # (C) 파일 저장
    # -------------------------------------------------
    if dataset:
        filename = f"history_data_final/nh_{vol_str}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(dataset, f, ensure_ascii=False, indent=4)
        print(f"   💾 저장 완료: {filename} (총 {len(dataset)}건)\n")
    else:
        print(f"   💨 수집된 데이터가 없습니다.\n")

print("🎉 대장정 완료! 고생하셨습니다!")

In [ ]:
import shutil
from google.colab import files
import os

# 1. 압축할 폴더 이름 (아까 코드에서 설정한 이름)
target_folder = "history_data_final"
output_filename = "korean_history_full_52vol"

# 폴더가 실제로 있는지 확인
if os.path.exists(target_folder):
    print(f"📦 '{target_folder}' 폴더를 '{output_filename}.zip'으로 압축 중...")

    # 2. zip 파일로 압축 (shutil 사용)
    shutil.make_archive(output_filename, 'zip', target_folder)

    print(f"✅ 압축 완료! ({output_filename}.zip)")
    print("⬇️ 다운로드를 시작합니다...")

    # 3. 로컬로 다운로드
    files.download(f"{output_filename}.zip")

else:
    print(f"❌ '{target_folder}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요!")

📦 'history_data_final' 폴더를 'korean_history_full_52vol.zip'으로 압축 중...
✅ 압축 완료! (korean_history_full_52vol.zip)
⬇️ 다운로드를 시작합니다...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>